Universidad Torcuato Di Tella

Metodos computacionales\
**Trabajo Práctico 1:
Resoluciones Numéricas de Ecuaciones Diferenciales**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def f_piecewise(x):
    out = np.zeros_like(x)
    m = (0 <= x) & (x < 0.25);  out[m] = 4.0*x[m]
    m = (0.25 <= x) & (x < 0.50); out[m] = -2.0*x[m] + 1.5
    m = (0.50 <= x) & (x < 0.75); out[m] =  2.0*x[m] - 0.5
    m = (0.75 <= x) & (x <= 1.00); out[m] = -4.0*x[m] + 4.0
    return out

def heat_explicit(alpha, f, dx, dt, T):
    N  = int(round(1.0/dx))
    x  = np.linspace(0.0, 1.0, N+1)
    Nt = int(round(T/dt))
    r  = alpha * dt / (dx*dx)
    U  = np.zeros((Nt+1, N+1))
    U[0, :] = f(x)
    for n in range(Nt):
        Un = U[n, :]
        U[n+1, 1:-1] = Un[1:-1] + r*(Un[2:] - 2.0*Un[1:-1] + Un[:-2])
    return U, x, np.linspace(0.0, T, Nt+1), r

def _thomas(a, b, c, d):
    n = len(b)
    ac, bc, cc, dc = a.copy(), b.copy(), c.copy(), d.copy()
    for i in range(1, n):
        m = ac[i-1] / bc[i-1]
        bc[i] -= m * cc[i-1]
        dc[i] -= m * dc[i-1]
    x = np.zeros(n)
    x[-1] = dc[-1] / bc[-1]
    for i in range(n-2, -1, -1):
        x[i] = (dc[i] - cc[i] * x[i+1]) / bc[i]
    return x

def heat_implicit(alpha, f, dx, dt, T):
    N  = int(round(1.0/dx))
    x  = np.linspace(0.0, 1.0, N+1)
    Nt = int(round(T/dt))
    r  = alpha * dt / (dx*dx)
    U  = np.zeros((Nt+1, N+1))
    U[0, :] = f(x)
    m = N - 1
    a = -r * np.ones(m-1)
    b = (1 + 2*r) * np.ones(m)
    c = -r * np.ones(m-1)
    for n in range(Nt):
        rhs = U[n, 1:-1].copy()
        U[n+1, 1:-1] = _thomas(a, b, c, rhs)
    return U, x, np.linspace(0.0, T, Nt+1), r

# Ejecución con los parámetros del TP
if __name__ == "__main__":
    alpha, dx, dt, T = 1.0, 0.05, 0.001, 0.1
    Ue, x, t, r = heat_explicit(alpha, f_piecewise, dx, dt, T)
    Ui, x, t, _ = heat_implicit(alpha, f_piecewise, dx, dt, T)
    # Guardado a CSV y GIF: ver notebook o script adjunto.



: 